In [1]:
#importo las librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image

In [2]:
#importo las librearias de Scikit Learn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor as KN
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

from sklearn.model_selection import GridSearchCV

In [3]:
# importo el dataset de Train y test
Train = pd.read_csv(r'Xy_train.csv', delimiter=',', parse_dates = True)
xtest_prueba =  pd.read_csv(r'X_test.csv', delimiter=',', parse_dates = True)

In [4]:
# muestro una lineas del dataset de train
Train.head(3)

,X,y
0,2.273360,6.054685
1,3.167583,4.581428
2,7.973655,5.392507


In [5]:
# muestro una lineas del dataset de test
xtest_prueba.head()

,X
0,6.170800
1,6.302022
2,8.689293
3,2.376897
4,2.707320


In [6]:
#busco si contienen nulos
Train.isnull().sum()

X    0
y    0
dtype: int64

In [7]:
xtest_prueba.isnull().sum()

X    0
dtype: int64

In [8]:
#veo la forma de los datasets
Train.shape

(100, 2)

In [9]:
xtest_prueba.shape

(20, 1)

#### Utilizo mis datos de train para probar distintos modelos y ver cual es el que mejor se adapta a los datos para luego hacer la prediccion con el xtest

In [10]:
#Divido mi dataset de train en x e y
x=Train.iloc[:,0:1]
y=Train.iloc[:,1:]

In [11]:
#divido mi x e y en train y test para entrenar a los modelos
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=5)

In [12]:
xtrain.shape

(80, 1)

In [13]:
xtest.shape

(20, 1)

In [14]:
#normalizo y escalo mis datos de train. auto scaling train- set (mean = 0, std = 1)
scaler = preprocessing.StandardScaler().fit(xtrain)
scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [15]:
# auto scalo mis muestras de train utilizando el scaler fiteado con el xtrain
xtrain_scal = scaler.transform(xtrain)
# auto scalo mis muestras de test utilizando el scaler fiteado con el xtrain
xtest_scal = scaler.transform(xtest) 

In [16]:
#Creo un data frame para guardar los resultados de los modelos
resultados = pd.DataFrame(columns=['Modelo','R2','MSE','MAE'])

#### Empiezo con un modelo de regresion lineal

In [17]:
#creo un modelo generico de regresion lineal
lr = LinearRegression()

In [18]:
#lo entreno el modelo con mis datos de train escalados
lr.fit(xtrain_scal, ytrain)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [19]:
#obtengo la prediccion
ypred_lr = lr.predict(xtest_scal)

In [20]:
##calculo los errores
r2_lr=np.sqrt(mean_squared_error(ytest, ypred_lr))
mse_lr=mean_squared_error(ytest, ypred_lr)
mae_lr=mean_absolute_error(ytest, ypred_lr)


In [21]:
resultados = resultados.append({'Modelo':'LR',
                                'R2':r2_lr,
                                'MSE':mse_lr,
                                'MAE':mae_lr},ignore_index=True)

In [22]:
resultados

,Modelo,R2,MSE,MAE
0,LR,2.005142,4.020593,1.498947


### Pruebo con SVR

In [23]:
#creo un modelo generico de SVR
SVR= SVR()

In [24]:
#lo entreno
SVR.fit(xtrain_scal, ytrain)

C:\Users\alcur\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [25]:
#realizo la prediccion
ypred_svr=SVR.predict(xtest_scal)

In [26]:
r2_svr=np.sqrt(mean_squared_error(ytest, ypred_svr))
mse_svr=mean_squared_error(ytest, ypred_svr)
mae_svr=mean_absolute_error(ytest, ypred_svr)
resultados = resultados.append({'Modelo':'SVR',
                                'R2':r2_svr,
                                'MSE':mse_svr,
                                'MAE':mae_svr},ignore_index=True)

In [27]:
resultados

,Modelo,R2,MSE,MAE
0,LR,2.005142,4.020593,1.498947
1,SVR,1.401304,1.963653,1.077236


### Pruebo con KNN

In [28]:
#Defino un modelo generico
KNN=KN()

In [29]:
#lo entreno
KNN.fit(xtrain_scal, ytrain)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [30]:
#realizo la prediccion
ypred_knn=KNN.predict(xtest_scal)

In [31]:
r2_knn=np.sqrt(mean_squared_error(ytest, ypred_knn))
mse_knn=mean_squared_error(ytest, ypred_knn)
mae_knn=mean_absolute_error(ytest, ypred_knn)
resultados = resultados.append({'Modelo':'KNN',
                                'R2':r2_knn,
                                'MSE':mse_knn,
                                'MAE':mae_knn},ignore_index=True)

In [32]:
resultados

,Modelo,R2,MSE,MAE
0,LR,2.005142,4.020593,1.498947
1,SVR,1.401304,1.963653,1.077236
2,KNN,0.966854,0.934807,0.821672


### Me quedo con el KNN ya que es el que menor MSE tiene
###### reliazo la prediccion con el modelo de KNN antes definido

In [33]:
xtest_scal_prueba = scaler.transform(xtest_prueba)

In [34]:
ypred_prueba=KNN.predict(xtest_scal_prueba)

In [35]:
ypred_prueba

array([[3.63663374],
       [3.13994297],
       [7.55531299],
       [5.62283126],
       [4.88806034],
       [3.45007417],
       [7.18494107],
       [1.36726338],
       [9.96576603],
       [4.8792717 ],
       [4.66558214],
       [3.20964464],
       [9.96576603],
       [2.28387358],
       [3.99515158],
       [6.25824741],
       [3.98545917],
       [3.57118882],
       [4.15059303],
       [3.13994297]])

### Pruebo de definir un nuevio modelo y entrenarlo con las 100 samples del dataset orignal de train para hacer la predicion con el dataset de test original

In [36]:
#defino mi x train y mi y train
xtrain_prueba = Train.iloc[:,0:1]
ytrain_prueba = Train.iloc[:,1:]

In [37]:
xtrain_prueba.shape

(100, 1)

In [38]:
ytrain_prueba.shape

(100, 1)

In [39]:
#normalizo y escalo mis datos de train. auto scaling train- set (mean = 0, std = 1)
scaler = preprocessing.StandardScaler().fit(xtrain_prueba)
scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [40]:
# auto scalo mis muestras de train utilizando el scaler fiteado con el xtrain
xtrain_scal_prueba = scaler.transform(xtrain_prueba)
xtest_scal_prueba = scaler.transform(xtest_prueba)

In [41]:
#Defino un modelo generico
KNN=KN()

In [42]:
#lo entreno con todo el dates de train esta vez
KNN.fit(xtrain_scal_prueba, ytrain_prueba)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [43]:
ypred_prueba=KNN.predict(xtest_scal_prueba)

In [44]:
ypred_prueba

array([[ 3.63663374],
       [ 3.13994297],
       [ 7.19019016],
       [ 5.47195654],
       [ 4.88806034],
       [ 3.85132562],
       [ 7.17995228],
       [ 1.36726338],
       [ 9.73789523],
       [ 5.16198951],
       [ 5.31746007],
       [ 3.20964464],
       [10.35309045],
       [ 2.18304688],
       [ 4.09851269],
       [ 6.69289768],
       [ 4.38469541],
       [ 3.57118882],
       [ 4.15059303],
       [ 3.13994297]])

### No se si conviene crear un nuevo modelo de KNN o usar el anterior, hay diferencia en la prediccion